# Toric Geometry

* Authors: Martin Bies, Luca Remke
* Version: OSCAR version 0.14.0
* Last modified: January 26, 2024

This tutorial provides an introduction to (normal) toric varieties in OSCAR. The aim is to describe important functionality and to exemplify it, so that users can start to use OSCAR more easily. Still, let us note that this tutorial does not aim to cover all of the existing functionality. Instead, we present a selection of arguably very important functions and features. For a more complete overview, please consult the documentation at https://docs.oscar-system.org/stable/AlgebraicGeometry/ToricVarieties/intro/.

For this tutorial, the reader is assumed to be familiar with OSCAR's commutative algebra and polyhedral geometry methods.

The content of this notebook is as follows:
1. [Affine toric varieties](#1-Affine-toric-varieties)
    1. [Constructing affine toric varieties](#1.A-Constructing-affine-toric-varieties)
    2. [Properties and attributes](#1.B-Properties-and-attributes)
2. [The projective space](#2-The-projective-space)
   1. [Constructing the projective space ](#2.A-Constructing-the-projective-space)
   2. [Properties and attributes](#2.B-Properties-and-attributes)
   3. [Weil and Cartier divisors](#2.C-Weil-and-Cartier-divisors)
   4. [Divisor classes](#2.D-Divisor-classes)
   5. [Line bundles](#2.E-Line-bundles)
   6. [Line bundle cohomology with cohomCalg](#2.F-Line-bundle-cohomology-with-cohomCalg)
   7. [The polyhedral fan](#2.G-The-polyhedral-fan)
3. [More general constructions of normal toric varieties](#3-More-general-constructions-of-normal-toric-varieties)
   1. [The projective space from its fan](#3.A-The-projective-space-from-its-fan)
   2. [Del Pezzo surfaces](#3.B-Del-Pezzo-surfaces)
   3. [Hirzebruch surfaces](#3.C-Hirzebruch-surfaces)
   4. [Weighted projective spaces](#3.D-Weighted-projective-spaces)
   5. [Affine spaces](#3.E-Affine-spaces)
   6. [Cyclic quotient singularities](3.F-Cyclic-quotient-singularities)
   7. [Toric blowups](#3.G-Toric-blowups)
   8. [Cartesian products](#3.H-Cartesian-products)
   9. [Triangulations](#3.I-Triangulations)
   10. [Benchmarking: An involved triangulation computation](#3.J-Benchmarking:-An-involved-triangulation-computation)
   11. [Turning off input checks](#3.K-Turning-off-input-checks)
4. [Advanced topics](#4-Advanced-topics)
   1. [Vanishing sets](#4.A-Vanishing-sets)
   2. [Intersection theory](#4.B-Intersection-theory)
   3. [Toric morphisms](#4.C-Toric-morphisms)
5. [Toric schemes and subvarieties](#5-Toric-schemes-and-subvarieties)

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.14.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2024 by The OSCAR Development Team


# 1 Affine toric varieties

We begin by discussing affine toric varieties. Those varieties are encoded by rational polyhedral cones, as stated in theorem 1.2.18 of the book *Toric Varieties* by David Cox, John Little and Hal Schenck:

Let $\sigma \subseteq N_{\mathbb{R}} \cong \mathbb{R}^n$ be a rational polyhedral cone with semigroup $S_\sigma = \sigma^\vee \cap M$. Then the associated affine toric variety $U_\sigma$ is given by

$$ U_\sigma = \mathrm{Spec} \left(\mathbb{C} \left[ S_\sigma \right]\right) = \mathrm{Spec} \left(\mathbb{C} \left[ \sigma^\vee \cap M \right]\right) \, .$$

In particular, it holds

$$\mathrm{dim}( U_\sigma ) = n \, \Leftrightarrow \, \text{the torus of $U_\sigma$ is } T_N = N \otimes_{\mathbb{Z}} \mathbb{C}^\ast \, \Leftrightarrow \, \sigma \text{ is strongly convex.}$$

We focus on *strongy convex* polyhedral cones $\sigma$, so that $T_N$ is the algebraic torus of $U_\sigma$.

## 1.A Constructing affine toric varieties

The following code constructs the affine space $\mathbb{C}^2$ as toric variety:

In [2]:
C = positive_hull([1 0; 0 1])
antv = affine_normal_toric_variety(C)

Normal toric variety

## 1.B Properties and attributes

The cone $\sigma$ encodes a lot of information about the affine toric variety $U_\sigma$. In the background, *OSCAR* conducts such investigations in polyhedral geometry with Polymake (cf. https://polymake.org/doku.php/start). This allows us to access a lot of properties of affine toric varieties.

The following support our claim that the toric variety *antv* constructed above is nothing but the affine space $\mathbb{C}^2$:

In [3]:
is_smooth(antv)

true

In [4]:
is_complete(antv)

false

In [5]:
is_affine(antv)

true

To prove that *antv* is $\mathbb{C}^2$, we can seek polynomials that cut out this affine variety in $\mathbb{C}^k$ for suitable integer $k$. In the case at hand, the polynomial ideal to be investigated is the toric ideal. We compute it as follows:

In [6]:
I = toric_ideal(antv)

ideal(0)

In [7]:
parent(gens(I)[1])

Multivariate polynomial ring in 2 variables x1, x2
  over rational field

This means, that the variety *antv* is cut out by the trivial ideal in $\mathbb{C}^2$. In other words, *antv* indeed just happens to be $\mathbb{C}^2$.

For a more interesting example, we consider the rational normal cone of degree 2:

In [8]:
C2 = positive_hull([1 1; -1 1])
antv2 = affine_normal_toric_variety(C2)

Normal toric variety

This variety is not smooth, but has a singularity. This we verify in *OSCAR* as follows:

In [9]:
is_smooth(antv2)

false

In [10]:
is_simplicial(antv2)

true

Just as above we can identify this affine variety more concretely by investigating its toric ideal:

In [11]:
I2 = toric_ideal(antv2)

ideal(-x1*x2 + x3^2)

In [12]:
parent(gens(I2)[1])

Multivariate polynomial ring in 3 variables x1, x2, x3
  over rational field

So in $\mathbb{C}^3$ with coordinate ring $\mathbb{C}[ x_1, x_2, x_3 ]$ we are looking at the variety $X = V( - x_1 x_2 + x_3^2  )$. This is a rather classical example of a simplicial, affine toric variety.

Similarly, we can consider the following affine toric variety:

In [13]:
C3 = positive_hull([2 -1; 0 1])
antv3 = affine_normal_toric_variety(C)

Normal toric variety

The cone *C3* is different from *C2*, yet *antv3* is isomorphic to *antv2*:

In [14]:
I3 = toric_ideal(antv3)

ideal(0)

In [15]:
parent(gens(I3)[1])

Multivariate polynomial ring in 2 variables x1, x2
  over rational field

# 2 The projective space

The most general (normal) toric varieties are encoded by collections of strongly convex polyhedral cones. And of course, it must not be a "random" collection of cones, but the cones have to be "compatible" with each other. Specifically, the collection in question must form a polyhedral fan. We give details on how to create such fans and the corresponding toric varieties in section 3 of this tutorial. However, before we discuss those advaned varieties, we wish to demonstrate more of the existing toric functionality in *OSCAR* by studying the well-known case of the projective space. For concreteness, we focus on the projective space $\mathbb{P}^2$.

## 2.A Constructing the projective space

We construct the projective space as follows:

In [16]:
P2 = projective_space(NormalToricVariety, 2)

Normal toric variety

Notice that the constructor expects us to state as first argument *NormalToricVariety*. This is a design choice made after careful consideration. Namely, the projective space could in principle be constructed as a scheme (with an arbitrary coefficient ring) or merely as a space parametrized by homogeneous coordinates. Depending on which of these perspectives is desired, specialized algorithms may be available.

As of February 2024, a lot of specialized toric functionality exists in OSCAR. In this regard, providing *NormalToricVariety* as the first argument not only ensures that we obtain the projective space as toric variety, but also that we can perform a large number of computations with it.

## 2.B Properties and attributes

Before we proceed, let us conduct a few consistency checks. If *P2* truely is the projective space of dimension 2, then is cannot be affine, but must be both smooth and projective. Let us verify this with *OSCAR*:

In [17]:
is_affine(P2)

false

In [18]:
is_smooth(P2)

true

In [19]:
is_projective(P2)

true

Likewise, *P2* must not have a torus factor. Let us elaborate a bit before we verify this.

Toric varieties contain the set $\mathbb{C}^\ast$ to a suitable power as dense open set. $\mathbb{C}^\ast$ (to some power) is called the algebraic torus, and this gives toric varieties their name. If *P2* had a torus factor, it would mean that *P2* factors into two varieties of which one is isomorphic to an algebraic torus. This is clearly not the case for *P2*, as we can verify with *OSCAR*:

In [20]:
has_torusfactor(P2)

false

Another basic, yet important, information about *P2* is its dimension. This should be 2 if *P2* is the projective space of dimension 2:

In [21]:
dim(P2)

2

In addition, we should expect that the projective space $\mathbb{P}^2$ is covered by three affine open subsets:

In [22]:
affine_open_covering(P2)

3-element Vector{AffineNormalToricVariety}:
 Normal toric variety
 Normal toric variety
 Normal toric variety

To conduct more interesting tests and computations with *P2*, let us next discuss divisors.

## 2.C Weil and Cartier divisors

Affine schemes/varieties come with a coordinate that tells us a lot of information about these varieties. For normal toric varieties - even if not affine - there exists a counterpart to this ring, the so-called Cox ring (often also called the homogeneous coordinate ring):

In [23]:
cox_ring(P2)

Multivariate polynomial ring in 3 variables over QQ graded by 
  x1 -> [1]
  x2 -> [1]
  x3 -> [1]

Just as for affine varieties/schemes, it makes sense in a toric variety to talk about $\mathbb{V}(x1)$, $\mathbb{V}(x2)$ and $\mathbb{V}(x3)$. These are Weil divisors. Over the integer $\mathbb{Z}$, these three divisors generate an important class of divisors on $\mathbb{P}^2$: The so-called *torus-invariant* Weil divisors.

We briefly mention that the algebraic torus $\mathbb{C}^\ast$ (to some power) is not only a dense open set of any toric variety, but also provides a torus action on the toric variety. The interested reader can find details about this in the book *Toric Varieties* by David Cox, John Little and Hal Schenck.

For this tutorial, we suffice it to conclude from the above that the group of torusinvariant Weil divisors on $\mathbb{P}^2$ is $\mathbb{Z}^3$. Let us now compute this group for *P2*:

In [24]:
torusinvariant_weil_divisor_group(P2)

GrpAb: Z^3

Divisors are linearly equivalent if their difference is a principal divisor, i.e. the divisor of a rational function. For the projective space $\mathbb{P}^2$, we therefore usually only talk about the hyperplane class. Put yet differently, we should expect that
* $D_1 = \mathbb{V}(x1)$,
* $D_2 = \mathbb{V}(x2)$,
* $D_3 = \mathbb{V}(x3)$,
  
are linearly equivalent. To see this, we can map torus-invariant Weil divisors to the class group $\mathrm{Cl}( \mathbb{P}^2 )$, which factors exactly by the linear equivalence of the torusinvariant Weil divisors.

In [25]:
class_group(P2)

GrpAb: Z

In [26]:
f1 = map_from_torusinvariant_weil_divisor_group_to_class_group(P2)

Map
  from GrpAb: Z^3
  to GrpAb: Z

To see the mapping prescription, execute the following:

In [27]:
matrix(f1)

[1]
[1]
[1]

This means $[D_1] = [D_2] = [D_3]$. Or put differently, the Weil divisors $D_1$, $D_2$, $D_3$ are linearly equivalent: $D_1 \sim D_2 \sim D_3$.

Similar functionality also exist for the group of torus-invariant Cartier divisors $\mathrm{Div}_T(\mathbb{P}^2)$. In the case of $\mathbb{P}^2$ those maps are a bit boring, essentially because $\mathbb{P}^2$ is smooth: Every toric variety in *OSCAR* is normal and so every Cartier divisor is a Weil divisor. Conversely, since $\mathbb{P}^2$ is smooth, every Weil divisor is Cartier and the notions coincide.

For non-smooth toric varieties, the notions will be different. With such spaces in mind, let us briefly demonstrate the corresponding functionality.

In [28]:
torusinvariant_cartier_divisor_group(P2)

GrpAb: Z^3

In [29]:
f2 = map_from_cartier_divisor_group_to_picard_group(P2)

Map
  from GrpAb: Z^3
  to GrpAb: Z

In [30]:
matrix(f2)

[1]
[1]
[1]

In [31]:
picard_group(P2)

GrpAb: Z

We can of course compute a map that take the torus-invariant Cartier divisors to the torus-invariant Weil divisors:

In [32]:
f3 = map_from_torusinvariant_cartier_divisor_group_to_torusinvariant_weil_divisor_group(P2)
matrix(f3)

[1   0   0]
[0   1   0]
[0   0   1]

At times, it will be interesting to find a torus-invariant Weil divisor $D$ that represents a certain divisor class. This is possible because $f_1$ is surjective:

In [33]:
is_surjective(f1)

true

Hence, for every $x \in \mathrm{Cl}(\mathbb{P}^2)$, there is a $y \in \mathrm{Div}_T(\mathbb{P}^2)$ with $f_1(y) = x$. We can compute such preimages:

In [34]:
[preimage(f1,x) for x in gens(class_group(P2))]

1-element Vector{GrpAbFinGenElem}:
 Element of
GrpAb: Z^3
with components [1 0 0]

Alternatively, we can even find the preinverse of $f_1$ directly:

In [35]:
matrix(preinverse(f1))

[1   0   0]

This functionality is particularly useful in the context of line bundles on toric varieties, i.e. the elements of the Picard group $\mathrm{Pic}(\mathbb{P}^2)$. We will turn to line bundles momentarily.

We complete this section by briefly mentioning the character lattice $N(\mathbb{P}^2)$. Since $\mathbb{P}^2$ has no torus factor, there is a short exact sequence:

$$0 \to N(\mathbb{P}^2) \hookrightarrow \mathrm{Div}_T( \mathbb{P}^2 ) \twoheadrightarrow \mathrm{Cl} ( \mathbb{P}^2 ) \to 0.$$

The map $N(\mathbb{P}^2) \hookrightarrow \mathrm{Div}_T( \mathbb{P}^2 )$ can be computed as follows:

In [36]:
f4 = map_from_character_lattice_to_torusinvariant_weil_divisor_group(P2)

Map
  from GrpAb: Z^2
  to GrpAb: Z^3

In [37]:
is_injective(f4)

true

In [38]:
is_surjective(f4)

false

In [39]:
matrix(f4)

[1   0   -1]
[0   1   -1]

The columns of this matrix are the ray generators of the fan of the projective space $\mathbb{P}^2$, that is the non-negative span of them defines a 1-dimensional cone (i.e. a ray) and these 1-dimensional cones are part of the polyhedral fan that defines $\mathbb{P}^2$. In fact, any cone in this fan is generated by a proper subset of these three ray generators. We will return to the polyhedral fan of $\mathbb{P}^2$ towards the end of section 2.

Before we discuss divisor classes, let us mention the following important divisors:

In [40]:
trivial_divisor(P2)

Torus-invariant, non-prime divisor on a normal toric variety

In [41]:
anticanonical_divisor(P2)

Torus-invariant, non-prime divisor on a normal toric variety

In [42]:
canonical_divisor(P2)

Torus-invariant, non-prime divisor on a normal toric variety

## 2.D Divisor classes

Let us now create a torus-invariant Weil divisor on the projective space $\mathbb{P}^2$. From the study of toric divisors above, we recall that the group of torus-invariant Weil divisors has 3 generators $D_1$, $D_2$ and $D_3$. Hence, the most general torus-invariant Weil divisor is of the form $a_1 D_1 + a_2 D_2 + a_3 D_3$ with $a_1, a_2, a_3 \in \mathbb{Z}$. The constructor for toric divisors on $\mathbb{P}^2$ expects the list $[a_1, a_2, a_3]$ as its second argument.

Let us now create the torusinvariant Weil divisor $D = 1 D_1 + 2 D_2 + 3 D_3$:

In [43]:
D = toric_divisor(P2,[1,2,3])

Torus-invariant, non-prime divisor on a normal toric variety

We support various properties and attributes for such a divisor:

In [44]:
is_ample(D)

true

In [45]:
is_very_ample(D)

true

In [46]:
is_cartier(D)

true

In [47]:
is_principal(D)

false

In [48]:
coefficients(D)

3-element Vector{ZZRingElem}:
 1
 2
 3

In particular we can compute the divsior class associated to this divisor:

In [49]:
DC = toric_divisor_class(D)

Divisor class on a normal toric variety

The task of finding a divisor in this divisor class is exactly the question of inverting the map $f_1$ introduced above. This is handled behind the scenes with the following command:

In [50]:
D2 = toric_divisor(DC)

Torus-invariant, non-prime divisor on a normal toric variety

Note that $D_2$ and $D$ are not identical:

In [51]:
D == D2

false

We can see this explicitly, by asking for the coefficients of $D_2$:

In [52]:
coefficients(D2)

3-element Vector{ZZRingElem}:
 6
 0
 0

However, $D \sim D_2$. This is the case since $D - D_2$ is a principal divisor:

In [53]:
is_principal(D - D2)

true

So, as expected $[D] = [D_2]$.

## 2.E Line bundles 

Line bundles are the elements of the Picard group $\mathrm{Pic}( \mathbb{P}^2 )$. They are created similarly to Weil divisor. Let us first demonstrate how we can compute the Picard group $\mathrm{Pic}(\mathbb{P}^2)$ in OSCAR:

In [54]:
picard_group(P2)

GrpAb: Z

Hence, this group has a single generator $G = \mathcal{O}_{\mathbb{P}^2}(1)$ and every element in this group is of the form $G^a = G \otimes \dots \otimes G = \mathcal{O}_{\mathbb{P}^2}(a)$ with $a \in \mathbb{Z}$. Therefore, the constructor for line bundles on the projective space expects $[a]$ as input. For instance, the following creates the line bundle $\mathcal{L} = \mathcal{O}_{\mathbb{P}^2}(1)$:

In [55]:
L = toric_line_bundle(P2,[1]) 

Toric line bundle on a normal toric variety

Again, various properties and attributes are supported for line bundles. For example, we can check if a line bundle is the trivial line bundle (i.e. the zero element in the Picard group):

In [56]:
is_trivial(L)

false

Other properties are accessed by computing a divisor (class) corresponding to the line bundle in question (i.e. inverting the map $\mathrm{CDiv}( \mathbb{P}^2 ) \to \mathrm{Pic}( \mathbb{P}^2 )$):

In [57]:
toric_divisor(L)

Torus-invariant, cartier, non-principal, prime divisor on a normal toric variety

In [58]:
toric_divisor_class(L)

Divisor class on a normal toric variety

Among others, ampleness of a line bundle is checked by checking it for the underlying divisor:

In [59]:
is_ample(L)

true

In [60]:
is_very_ample(L)

true

Another example is the degree of the line bundle:

In [61]:
degree(L)

1

Let us mention that toric line bundles can be tensorised via `*`. The `n`-th tensor product is calculated via `^n`. In particular, `^(-1)` calculates the inverse of a line bundle. Alternatively, the inverse can also be calculated via `inv`.

The following lines bundles are of particular importance:

In [62]:
structure_sheaf(P2)

Toric line bundle on a normal toric variety

In [63]:
anticanonical_bundle(P2)

Toric line bundle on a normal toric variety

In [64]:
canonical_bundle(P2)

Toric line bundle on a normal toric variety

## 2.F Line bundle cohomology with cohomCalg

To compute line bundle cohomology, we use the cohomCalg algorithm (cf. https://github.com/BenjaminJurke/cohomCalg). It is executed as follows:

In [65]:
all_cohomologies(L)

3-element Vector{ZZRingElem}:
 3
 0
 0

This means that $h^0( \mathbb{P}^2, \mathcal{L} ) = 3$ and $h^1( \mathbb{P}^2, \mathcal{L} ) = h^2( \mathbb{P}^2, \mathcal{L} ) = 0$. This result is of course well-known to the experts. This functionality works as long as the variety in question is both smooth and complete. It will also work if the variety is both simplicial and projective.

Of course, we can also compute just some of the line bundle cohomologies. For instance, the following finds $h^0( \mathbb{P}^2, \mathcal{L} )$:

In [66]:
cohomology(L,0)

3

The Cox ring of the toric variety is very important. It is the algebraic counterpart to the structure sheaf $\mathcal{O}_{\mathbb{P}^2}$:

In [67]:
cox_ring(P2)

Multivariate polynomial ring in 3 variables over QQ graded by 
  x1 -> [1]
  x2 -> [1]
  x3 -> [1]

As such, it comes at no surprise that a basis of the global sections of line bundles can be found from the homogeneous components of the Cox ring. An alternative methods exists via Laurent monomials. Both options are available within OSCAR:

In [68]:
basis_of_global_sections_via_homogeneous_component(L)

3-element Vector{MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}}:
 x3
 x2
 x1

In [69]:
basis_of_global_sections_via_rational_functions(L)

3-element Vector{MPolyQuoRingElem{QQMPolyRingElem}}:
 x1_
 x2*x1_
 1

Note that in this expression *x1_* refers to $\frac{1}{x_1}$. We can see this as follows:

In [70]:
parent(basis_of_global_sections_via_rational_functions(L)[1])

Quotient
  of multivariate polynomial ring in 4 variables x1, x2, x1_, x2_
    over rational field
  by ideal(x1*x1_ - 1, x2*x2_ - 1)

For convenience, by default we use the homogeneous components:

In [71]:
basis_of_global_sections(L)

3-element Vector{MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}}:
 x3
 x2
 x1

## 2.G The polyhedral fan

We now come back to constructing more general toric varieties. As mentioned above, this in general relies on the construction of a polyhedral fan consisting of strongly convex polyhedral cones. This polyhedral geometry is the backbone for many computations in toric geometry. Of course, *OSCAR* knows the fan of the projective space $\mathbb{P}^2$:

In [72]:
fan_of_P2 = polyhedral_fan(P2)

Polyhedral fan in ambient dimension 2

You can plot this fan by invoking the command *visualize(f)*. This will (most likely) open a separate window with an illustration of the fan. Alternatively, we can investigate the fan in more details by computing its rays and maximal cones:

In [73]:
rays(fan_of_P2)

3-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [0, 1]
 [-1, -1]

In [74]:
maximal_cones(fan_of_P2)

3-element SubObjectIterator{Cone{QQFieldElem}}:
 Polyhedral cone in ambient dimension 2
 Polyhedral cone in ambient dimension 2
 Polyhedral cone in ambient dimension 2

In [75]:
[rays(c) for c in maximal_cones(fan_of_P2)]

3-element Vector{SubObjectIterator{RayVector{QQFieldElem}}}:
 [[1, 0], [0, 1]]
 [[0, 1], [-1, -1]]
 [[1, 0], [-1, -1]]

Indeed, this is the famous fan of the projective space $\mathbb{P}^2$.

# 3 More general constructions of normal toric varieties

## 3.A The projective space from its fan

We now want to demonstrate how a toric variety can be created from a fan. As we have gained familarity with the projective space in the previous sections, it is natural to exemplify the operations on the projective space $\mathbb{P}^2$.

In OSCAR, a normal toric variety can be created from a polyhedral fan in a number of ways. This includes the following:
- Specify the maximal cones `max_cones` and the ray generators `rays` of the fan.
- Construct the polyhedral fan `PF` and use it as input for the toric variety.
- Specify a polyhedron `P`. Internally, we then create the normal fan of this polyhedron and return the corresponding normal toric variety.

Let us use the first method. The interested reader can find information on the other two approaches in the documentation.

In [76]:
ray_generators = [[1, 0], [0, 1], [-1, -1]]
max_cones = [[1, 2], [2, 3], [3, 1]]
X = normal_toric_variety(max_cones, ray_generators)

Normal toric variety

Let us again conduct a few consistency checks to gain confidence that this variety indeed is the projective space $\mathbb{P}^2$:

In [77]:
is_projective(X)

true

In [78]:
is_smooth(X)

true

In [79]:
dim(X)

2

In [80]:
class_group(X)

GrpAb: Z

In [81]:
torusinvariant_weil_divisor_group(X)

GrpAb: Z^3

In [82]:
cox_ring(X)

Multivariate polynomial ring in 3 variables over QQ graded by 
  x1 -> [1]
  x2 -> [1]
  x3 -> [1]

## 3.B Del Pezzo surfaces 

We provide a constructor for del Pezzo surfaces. For instance, we can issue:

In [83]:
dP3 = del_pezzo_surface(3)

Normal toric variety

To demonstrate that the line bundle cohomology computations also work in such more general settings, let us construct a toric line bundle and compute its sheaf cohomology dimensions:

In [84]:
picard_group(dP3)

GrpAb: Z^4

In [85]:
L2 = toric_line_bundle(dP3,[3,2,1,0])

Toric line bundle on a normal toric variety

In [86]:
all_cohomologies(L2)

3-element Vector{ZZRingElem}:
 6
 0
 0

It is worth recalling that we can find a basis of the global sections. However, a basis for the higher sheaf cohomologies cannot (yet) be computed with OSCAR.

In [87]:
basis_of_global_sections(L2)

6-element Vector{MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}}:
 x2^2*x3*e3^3
 x1*x2*x3*e2*e3^2
 x1*x2^2*e1*e3^2
 x1^2*x3*e2^2*e3
 x1^2*x2*e1*e2*e3
 x1^3*e1*e2^2

## 3.C Hirzebruch surfaces

Similarly, we can also construct Hirzebruch surfaces:

In [88]:
F2 = hirzebruch_surface(2)

Normal toric variety

Of course, we can create line bundles on this surface and investigate their properties:

In [89]:
l = ToricLineBundle(F2, [0,1])
(is_trivial(l), is_basepoint_free(l), is_ample(l), is_very_ample(l))

(false, false, false, false)

## 3.D Weighted projective spaces

There is also support for weighted projective spaces in *OSCAR*:

In [90]:
P231 = weighted_projective_space(NormalToricVariety, [2,3,1])

Normal toric variety

In [91]:
is_smooth(P231)

false

In [92]:
is_simplicial(P231)

true

In [93]:
is_projective(P231)

true

Consequently the necessary conditions are satisfied to apply *cohomCalg* to compute line bundle cohomology on this space:

In [94]:
l = toric_line_bundle(P231, [3])
all_cohomologies(l)

3-element Vector{ZZRingElem}:
 37
 0
 0

## 3.E Affine spaces

Likewise, affine spaces are supported:

In [95]:
A5 = affine_space(NormalToricVariety, 5)

Normal toric variety

In [96]:
is_affine(A5)

true

In [97]:
dim(A5)

5

## 3.F Cyclic quotient singularities

Cyclic quotient singularities are quotients of $\mathbb{C}^2$ by the action of $\mathbb{Z}/n\mathbb{Z}$ acting via 
\begin{equation*}
    \begin{pmatrix} \xi & 0 \\ 0 & \xi^q \end{pmatrix},
\end{equation*}
where $\xi$ is an $n$-th root of unity and $q,n$ are coprime integers,such that $0 < q < n$. Here is an example:

In [98]:
cqs = cyclic_quotient_singularity(7,5)

Cyclic quotient singularity Y(7, 5)

In [99]:
dim(cqs)

2

Note that in the constructor `cyclic_quotient_singularity(n, q)`, we must have $0<q<n$ and $q,n$ coprime.

We can then compute the Hirzebruch-Jung continued fraction corresponding to $n/q$:

In [100]:
cf = continued_fraction_hirzebruch_jung(cqs)

3-element Vector{ZZRingElem}:
 2
 2
 3

There is a rational number corresponding to a Hirzebruch-Jung continued fraction $[c_1,c_2,...,c_n]$. This rational number is 
\begin{equation*}
r([c_1,c_2,...,c_n]) = c_1 - \tfrac{1}{r([c_2,...,c_n])},
\end{equation*}
where $r([c_n]) = c_n$. Note that this is differs in sign from what is commonly known as continued fraction. In the above case we thus have:

In [101]:
r = cf[1] - 1//(cf[2] - (1//cf[3]))

7//5

This result we can also compute with the following function:

In [102]:
continued_fraction_hirzebruch_jung_to_rational(cf)

7//5

This also works the other way around, i.e. we can seek the continued fraction corresponding to a rational number:

In [103]:
cf2 = rational_to_continued_fraction_hirzebruch_jung(QQFieldElem(7,5))

3-element Vector{ZZRingElem}:
 2
 2
 3

In [104]:
cf == cf2

true

Here is a more entertaining example:

In [105]:
r = QQFieldElem(2464144958, 145732115)
cf = rational_to_continued_fraction_hirzebruch_jung(r)

7-element Vector{ZZRingElem}:
 17
 11
 23
 46
 18
 19
 37

In [106]:
continued_fraction_hirzebruch_jung_to_rational(cf)

2464144958//145732115

Finally, we can also compute the Hirzebruch-Jung continued fraction corresponding to $q/(n-q)$:

In [107]:
dcf = dual_continued_fraction_hirzebruch_jung(cqs)

2-element Vector{ZZRingElem}:
 4
 2

In [108]:
continued_fraction_hirzebruch_jung_to_rational(dcf)

7//2

## 3.G Toric blowups

By means of a star subdivision, one can conduct a toric blowup. The relevant functionality is available in OSCAR. To this end let us look at the projective space anew:

In [109]:
P2

Normal, non-affine, smooth, projective, 2-dimensional toric variety without torusfactor

In [110]:
cox_ring(P2)

Multivariate polynomial ring in 3 variables over QQ graded by 
  x1 -> [1]
  x2 -> [1]
  x3 -> [1]

Let us now perform a blowup of the locus $V( x_1, x_2 )$. This is done as follows:

In [111]:
BP2 = blowup_on_ith_minimal_torus_orbit(P2, 1, "e")

Toric blowdown morphism

The number "1" selects the maximal cone of the fan of $\mathbb{P}^2$ which we divide by the star subdivision. The string "e" defines the name for the new homogeneous coordinate of the toric variety. Here we have the following correspondence between ray generators and homogeneous variables:
\begin{align}
x_1 &\leftrightarrow [1,0] \,\\
x_2 &\leftrightarrow [0,1] \,\\
x_3 &\leftrightarrow [-1,-1] \,.
\end{align}
The blowup thus requires a star subdivision of the cone $C = \mathrm{Span}_{\mathbb{Z}_{\geq 0}} \left\{ [1,0], [0,1]\right\}$. To this end, we first compute the sum of the generators of $C$. This is a new ray generator:
$$e \leftrightarrow [1,1] \,,$$
which we use to define the cones which maximally subdivide $C$, namely:
$$C_1 = \mathrm{Span}_{\mathbb{Z}_{\geq 0}} \left\{ [1,0], [1,1]\right\} \, , \qquad C_2 = \mathrm{Span}_{\mathbb{Z}_{\geq 0}} \left\{ [1,1], [0,1]\right\} \, .$$
Let us see that this is indeed the case for the above computed toric space $B\mathbb{P}^2$:

In [112]:
fan_of_BP2 = polyhedral_fan(domain(BP2))

Polyhedral fan in ambient dimension 2

In [113]:
rays(fan_of_BP2)

4-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [0, 1]
 [-1, -1]
 [1, 1]

So we notice the new ray $[1,1]$. To find the new cones $C_1$ and $C_2$, we issue the following:

In [114]:
[rays(c) for c in maximal_cones(fan_of_BP2)]

4-element Vector{SubObjectIterator{RayVector{QQFieldElem}}}:
 [[1, 0], [1, 1]]
 [[0, 1], [1, 1]]
 [[0, 1], [-1, -1]]
 [[1, 0], [-1, -1]]

So indeed, the last two cones are the expected new cones. The first two are the cones that exist for the fan of $\mathbb{P}^2$ and were not changed.

The variable $e$ we can see most prominently in the Cox ring:

In [115]:
cox_ring(domain(BP2))

Multivariate polynomial ring in 4 variables over QQ graded by 
  x1 -> [1 0]
  x2 -> [1 0]
  x3 -> [0 1]
  e -> [-1 1]

It is instructive to look at the following alternative grading of the Cox ring (and we order the variables in ascending order):
$$\begin{array}{ccc|c} x_1 & x_2 & x_3 & e \\ \hline 1 & 1 & 1 & 0 \\ 1 & 1 & 0 & -1 \end{array}$$
In the first line you notice the ordinary grading of the Cox ring of $\mathbb{P}^2$. The second line corresonds to the toric blowup that has been conducted. The $-1$ for $e$ tells us that this is the homogeneous coordinate introduced to resemble the blowup $\mathbb{P}^1$. The 1 for $x_1$ and $x_2$ tells us that this blowup added a $\mathbb{P}^1$ at the locus $\{ x_1 = x_2 = 0\}$ in $\mathbb{P}^2$. In other words, in $B\mathbb{P}^2$, $x_1$ and $x_2$ cannot vanish simultaneously:
$$\{ [x_1 : x_2 : x_3 : e] \in B\mathbb{P}^2 | x_1 = x_2 = 0\} = \emptyset \, .$$
It is instructive to recall that the generators of the Stanley-Reisner ideal encode exactly which homogeneous variables cannot vanish simultaneously:

In [116]:
stanley_reisner_ideal(domain(BP2))

ideal(x1*x2, x3*e)

This means that $\emptyset = V( x_2, x_1 ) \subseteq B \mathbb{P}^2$ and $\emptyset = V( x_3, e ) \subset B \mathbb{P}^2$.

Along these lines, we can also see that the locus $V(e) \subset B\mathbb{P}^2$ is a $\mathbb{P}^1$. Namely, from the Stanley-Reisner ideal we learn that $x_3$ and $e$ cannot vanish simultaneously. So $x_3 \neq 0$ and we can rescale it to $1$. This rescaling effectively eliminates the 2nd line in the grading of the Cox ring and leaves us with
$$\begin{array}{ccc|c} x_1 & x_2 \\ \hline 1 & 1 \end{array}$$
So we notice, that the variables $x_1$ and $x_2$ each enjoy the standard grading $(1,1)$. Also, since $x_1 x_2 \in I_{\text{SR}}( B\mathbb{P}^2 )$, it holds $\emptyset = V( x_2, x_1 ) \subset B \mathbb{P}^2$. This implies $V( e ) \cong \mathbb{P}^1_{[x_1 \colon x_2]}$.

## 3.H Cartesian products

It is also possible to compute Cartesian products of toric varieties. Here is an example:

In [117]:
product = P2 * F2

Normal toric variety

In [118]:
cox_ring(product)

Multivariate polynomial ring in 7 variables over QQ graded by 
  xx1 -> [1 0 0]
  xx2 -> [1 0 0]
  xx3 -> [1 0 0]
  yt1 -> [0 1 0]
  yx1 -> [0 0 1]
  yt2 -> [0 1 0]
  yx2 -> [0 2 1]

Notice that the set of variables "xx*" correspond to the $\mathbb{P}^2$ factor, and the remaining variables are the homogeneous coordinates of the Hirzebruch surface $\mathbb{F}_2$.

## 3.I Triangulations

Triangulations of polytopes open an entire arena for creating families of toric varieties whose properties are often related to one-another in interesting ways. A prominent example along these lines is the Kreuzer-Skarke database (cf. http://hep.itp.tuwien.ac.at/~kreuzer/CY/). Another popular example is the conifold transition, which we now use to demonstrate triangulation functionality in OSCAR:

In [119]:
P = convex_hull([0 0 0; 0 0 1; 1 0 1; 1 1 1; 0 1 1])

Polyhedron in ambient dimension 3

In [120]:
 (v1, v2) = normal_toric_varieties_from_star_triangulations(P)

2-element Vector{NormalToricVariety}:
 Normal toric variety
 Normal toric variety

If desired, you can look at $P$ by invoking *visualize(P)*. The polyhedron $P$ admits exactly two fine, regular, star triangulations. These were used to create the toric varieties $v_1$ and $v_2$. We now study these two varieties:

In [121]:
is_smooth(v1)

true

In [122]:
is_projective(v1)

false

In [123]:
dim(v1)

3

In [124]:
is_complete(v1)

false

In [125]:
cox_ring(v1)

Multivariate polynomial ring in 4 variables over QQ graded by 
  x1 -> [1]
  x2 -> [-1]
  x3 -> [-1]
  x4 -> [1]

In [126]:
stanley_reisner_ideal(v1)

ideal(x1*x4)

If we repeat these line with $v_2$, then the only difference is:

In [127]:
stanley_reisner_ideal(v2)

ideal(x2*x3)

So in a sence, the transition from $v_1$ to $v_2$ means that the roles of ($x_2$, $x_4$) and ($x_1$, $x_3$) are exchanged. Indeed, the conifold transition means to first perform a deformation which shrinks an algebraic cycle to zero. This leads to a conifold singularity which is subsequently resolved by blowing up another algebraic cycle.

## 3.J Benchmarking: An involved triangulation computation

Triangulations are a topic that features prominently in applied mathematics, e.g. in string theory. The following code was used in a string theory publication (https://arxiv.org/abs/2205.00008):

In [128]:
function involved_triangulation()
    
    # (0) Define the polytope
    fac1 = convex_hull( [-1 -1 -1; 2 -1 -1; -1 2 -1]);
    fac2 = convex_hull( [-1 -1 -1; 2 -1 -1; -1 -1 5]);
    fac3 = convex_hull( [-1 -1 -1; -1 2 -1; -1 -1 5]);
    fac4 = convex_hull( [2 -1 -1; -1 2 -1; -1 -1 5]);
    fac = fac1;
    
    # (1) Set variable names of our choice
    vars_dict = Dict();
    vars_dict[matrix(ZZ,[-1 -1 -1])] = "x0";
    vars_dict[matrix(ZZ,[2 -1 -1])] = "x1";
    vars_dict[matrix(ZZ,[-1 2 -1])] = "x2";
    vars_dict[matrix(ZZ,[-1 -1 5])] = "x3";
    vars_dict[matrix(ZZ,[-1 -1 0])] = "x4";
    vars_dict[matrix(ZZ,[-1 -1 1])] = "x5";
    vars_dict[matrix(ZZ,[-1 -1 2])] = "x6";
    vars_dict[matrix(ZZ,[-1 -1 3])] = "x7";
    vars_dict[matrix(ZZ,[-1 -1 4])] = "x8";
    vars_dict[matrix(ZZ,[-1 0 -1])] = "x9";
    vars_dict[matrix(ZZ,[-1 0 0])] = "x10";
    vars_dict[matrix(ZZ,[-1 0 1])] = "x11";
    vars_dict[matrix(ZZ,[-1 0 2])] = "x12";
    vars_dict[matrix(ZZ,[-1 0 3])] = "x13";
    vars_dict[matrix(ZZ,[-1 1 -1])] = "x14";
    vars_dict[matrix(ZZ,[-1 1 0])] = "x15";
    vars_dict[matrix(ZZ,[-1 1 1])] = "x16";
    vars_dict[matrix(ZZ,[0 -1 -1])] = "x17";
    vars_dict[matrix(ZZ,[0 -1 0])] = "x18";
    vars_dict[matrix(ZZ,[0 -1 1])] = "x19";
    vars_dict[matrix(ZZ,[0 -1 2])] = "x20";
    vars_dict[matrix(ZZ,[0 -1 3])] = "x21";
    vars_dict[matrix(ZZ,[0 0 -1])] = "x22";
    vars_dict[matrix(ZZ,[0 0 1])] = "x23";
    vars_dict[matrix(ZZ,[0 1 -1])] = "x24";
    vars_dict[matrix(ZZ,[1 -1 -1])] = "x25";
    vars_dict[matrix(ZZ,[1 -1 0])] = "x26";
    vars_dict[matrix(ZZ,[1 -1 1])] = "x27";
    vars_dict[matrix(ZZ,[1 0 -1])] = "x28";
    
    #(2) Form point set to be triangulated
    points = lattice_points(fac);
    d = ambient_dim(fac)
    pts = vcat(zero_matrix(ZZ, 1, d), matrix(ZZ, transpose(reduce(hcat,points))));
    
    #(3) Triangulate
    trias = star_triangulations(pts; full=true, regular=true);
    trias = [[[c[k]-1 for k in 2:length(c)] for c in t] for t in trias];
    
    #(4) Compute rays of all toric varieties
    fan_rays = matrix(ZZ, points);
    
    #(5) Construct ring which contains all Stanley-Reisner ideals
    max_cones = IncidenceMatrix(vcat(trias[1]));
    pmfan = Polymake.fan.PolyhedralFan(RAYS=fan_rays, MAXIMAL_CONES=max_cones);
    variety = NormalToricVariety(PolyhedralFan{fmpq}(pmfan));
    my_vars = [vars_dict[fan_rays[i,:]] for i in 1:nrows(fan_rays)];
    set_coordinate_names(variety, my_vars);
    R = cox_ring(variety);
    
    #(6) Compute all ideals
    sr_ideals = [];
    for n in 1:length(trias)
        id = stanley_reisner_ideal(R, SimplicialComplex(trias[n]));
        push!(sr_ideals,id);
    end;
    
    #(7) Finall intersect all ideals
    return common_sr_ideal = reduce(intersect,sr_ideals);
    
end

involved_triangulation (generic function with 1 method)

In a nutshell, this code computes all fine regular star triangulations of the polyhedron *fac1*, computes the Stanley-Reisner of the associated toric varieties and then intersects these ideals. As such, it finds the intersection theory that is common to this family of toric varieties.

The code for *fac1* and *fac4* will likely execute within a few seconds. For the polyhedrons *fac2* or *fac3*, the task is much more involved and will likely take a few minutes. This is why in the above code, we fixed the choice of polyhedron to *fac1*. Brave users may which to challenge their computer a bit by altering that choice to *fac2* or *fac3*. The computations should still complete on most private computers within a few minutes.

To benchmark the execution time of the function *involved_triangulation*, and thus provide comparision to other computer systems, we can execute the following:

In [129]:
using BenchmarkTools
@btime involved_triangulation()

  277.839 ms (823024 allocations: 30.45 MiB)


ideal(x24*x1, x17*x1, x2*x1, x0*x1, x2*x28, x9*x28, x14*x25, x0*x25, x17*x24, x9*x2, x0*x2, x0*x14, x9*x14*x1, x0*x24*x28, x14*x17*x28, x9*x24*x25, x2*x17*x25, x22*x25*x28*x1, x14*x22*x28*x1, x9*x22*x25*x1, x22*x24*x25*x28, x17*x22*x25*x28, x14*x22*x24*x28, x0*x17*x22*x28, x2*x22*x24*x25, x9*x17*x22*x25, x14*x2*x22*x24, x9*x14*x22*x24, x0*x9*x22*x24, x14*x2*x17*x22, x9*x14*x17*x22, x0*x9*x17*x22)

You might find it interesting to compare your executation time to the following result:
* Execution time: 264.934ms,
* 820981 allocations (30.42 MiB).

This result was obtained on January 17, 2024 by runnning the OSCAR (see the beginning of this notebook for the version) with Julia 1.9.2 on a TUXEDO InfinityBook Pro 14 v6 with Intel Core i7-Quad-Core, 64GB RAM and operating system Tuxedo OS (based on Ubuntu 22.04).

## 3.K Turning off input checks

OSCAR aims for a broad user audience. As such, input is usually checked for consistency. This comes at the expense of loosing a bit of performance and possibly modifying the input data (if needed), e.g. shuffling it. In the toric settings, such shufflings of e.g. the order of rays can be undesirable.

It is possible to switch off such input checks when creating toric varieties. *However, it must be emphasized that then it is the user's responsibility to provide sane input.*

As an example, the following lines are executed internally to create the del Pezzo surface $dP_3$:

In [130]:
fan_rays = [1 0; 0 1; -1 -1; 1 1; 0 -1; -1 0]
cones = IncidenceMatrix([[1, 4], [2, 4], [1, 5], [5, 3], [2, 6], [6, 3]])
variety = normal_toric_variety(PolyhedralFan(fan_rays, cones; non_redundant = true))

Normal toric variety

Notice the *non_redundant = true* entry in the constructor for *NormalToricVariety*. This switches off the input check by informing OSCAR that we (= the users) made sure that there is no redundancy in the input. It is encouraged to use this functionality with care and only after checking the input carefully.

To continue with the above example, we would like to see that this is indeed a del Pezzo surface. This can be seen directly from the fan - the three last rays correspond to the star subdivision of the 3 maximal cones of $\mathbb{P}^2$ - or by studying the properties of the variety in more detail. We opt for the latter and first assign suitable variable names:

In [131]:
vars = ["x1", "x2", "x3", "e1", "e2", "e3"]
 set_coordinate_names(variety, vars[1:6])

It is also convenient to make a (valid!) standard choice for the grading of the cox ring. Namely, this must be such that the grading matrix corresponds to the map from the character lattice to the torus-invariant Weil divisor group. It is not too hard to verify that the following is a valid choice:

In [132]:
weights = matrix(ZZ, [1 1 1 0; 1 1 0 1; 1 0 1 1; 0 -1 0 0; 0 0 -1 0; 0 0 0 -1])
set_attribute!(variety, :map_from_torusinvariant_weil_divisor_group_to_class_group, hom(torusinvariant_weil_divisor_group(variety), class_group(variety), weights))

When executing *del_pezzo_surface(3)* more properties are set by OSCAR. But that put aside, the above are the internal steps of the constructor.

With that said, the following should identify this space as the del Pezzo surface $dP_3$:

In [133]:
cox_ring(variety)

Multivariate polynomial ring in 6 variables over QQ graded by 
  x1 -> [1 1 1 0]
  x2 -> [1 1 0 1]
  x3 -> [1 0 1 1]
  e1 -> [0 -1 0 0]
  e2 -> [0 0 -1 0]
  e3 -> [0 0 0 -1]

In [134]:
is_smooth(variety)

true

In [135]:
is_projective(variety)

true

In [136]:
[betti_number(variety,i) for i in range(0,4)]

5-element Vector{ZZRingElem}:
 1
 0
 4
 0
 1

In [137]:
picard_group(variety)

GrpAb: Z^4

In [138]:
torusinvariant_weil_divisor_group(variety)

GrpAb: Z^6

In [139]:
character_lattice(variety)

GrpAb: Z^2

# 4 Advanced topics

## 4.A Vanishing sets

For (smooth, complete) and (simplicial, projective) toric varieties, it is possible to find a unified description for all line bundles for which a certain cohomology class vanishes. This set is then referred to as a vanishing set.

The first description of this approach can be found in appendix B of https://arxiv.org/pdf/1802.08860.pdf. The corresponding functionality is available in Oscar.

We begin by demonstrating this for the projective space $\mathbb{P}^2$, for which the vanishing sets are well-known. Namely:
\begin{align}
V^0( \mathbb{P}^2 ) &= \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | h^0(\mathbb{P}^2, \mathcal{L}) = 0\} = \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | \mathrm{deg}( \mathcal{L} ) < 0 \} \, , \\
V^1( \mathbb{P}^2 ) &= \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | h^1(\mathbb{P}^2, \mathcal{L}) = 0\} = \mathrm{Pic}(\mathbb{P}^2) \, , \\
V^2( \mathbb{P}^2 ) &= \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | h^2(\mathbb{P}^2, \mathcal{L}) = 0\} = \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | \mathrm{deg}( \mathcal{L} ) > -2 \}\, .
\end{align}
Let us recompute this with OSCAR:

In [140]:
vs = vanishing_sets(P2)

3-element Vector{ToricVanishingSet}:
 Toric vanishing set for cohomology indices [0]
 Toric vanishing set for cohomology indices [1]
 Toric vanishing set for cohomology indices [2]

In [141]:
polyhedra(vs[1])

1-element Vector{Polyhedron{QQFieldElem}}:
 Polyhedron in ambient dimension 1

In [142]:
p0 = polyhedra(vs[1])[1]

Polyhedron in ambient dimension 1

In [143]:
vertices(p0)

1-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0]

In [144]:
rays(p0)

1-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1]

We are thus looking at $C = \mathrm{Span}_{\mathbb{Z}_{\geq 0}}(1) \}$. This cone is a subset of $\mathrm{Pic}( \mathbb{P}^2)$ and the vanishing set is its complement, that is:
$$V^0( \mathbb{P}^2 ) = \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | h^0(\mathbb{P}^2, \mathcal{L}) = 0\} = \{ \mathcal{L} \in \mathrm{Pic}(\mathbb{P}^2) | \mathcal{L} ) \notin C \} \, .$$
It is readily verified that this indeed matches the above expectation.

Recall that $V^1( \mathbb{P}^2 ) = \mathrm{Pic}(\mathbb{P}^2)$. This we can see quickly:

In [145]:
polyhedra(vs[2])

Polyhedron{QQFieldElem}[]

This list of polyhedra is empty. In other words, the vanishing set is the complement of the empty set in $\mathrm{Pic}( \mathbb{P}^2 )$, which is nothing but the entire Picard group.

Let us now look at the vanishing set for $h^2$:

In [146]:
polyhedra(vs[3])

1-element Vector{Polyhedron{QQFieldElem}}:
 Polyhedron in ambient dimension 1

In [147]:
p3 = polyhedra(vs[3])[1]

Polyhedron in ambient dimension 1

In [148]:
vertices(p3)

1-element SubObjectIterator{PointVector{QQFieldElem}}:
 [-3]

In [149]:
rays(p3)

1-element SubObjectIterator{RayVector{QQFieldElem}}:
 [-1]

We are thus looking at $P_3 = \{-3\} + \mathrm{Span}_{\mathbb{Z}_{\geq 0}} \{-1\}$. Its complement is $V^2( \mathbb{P}^2 )$, which matches our expectation.

Recall that we constructed a line bundle $\mathcal{L}$ on $\mathbb{P}^2$ above:

In [150]:
L = toric_line_bundle(P2, [1])

Toric line bundle on a normal toric variety

Its cohomologies are:

In [151]:
all_cohomologies(L)

3-element Vector{ZZRingElem}:
 3
 0
 0

So $L$ is not $V^0( \mathbb{P}^2 )$ but in $V^1( \mathbb{P}^2 )$ and $V^2( \mathbb{P}^2 )$:

In [152]:
contains(vs[1],L)

false

In [153]:
contains(vs[2],L)

true

In [154]:
contains(vs[3],L)

true

## 4.B Intersection theory

Intersection theory for toric varieties is well-understood and the Chow ring under good control. For a good example, let us consider the del Pezzo surface $dP_2$:

In [155]:
dP2 = del_pezzo_surface(2)

Normal toric variety

Recall that this constructors invokes standard names for the homogeneous coordinates. This we can see upon execution of the Cox ring:

In [156]:
S = cox_ring(dP2)

Multivariate polynomial ring in 5 variables over QQ graded by 
  x1 -> [1 1 1]
  x2 -> [1 1 0]
  x3 -> [1 0 1]
  e1 -> [0 -1 0]
  e2 -> [0 0 -1]

We now recall the toric ideal-variety correspondence from proposition 5.2.7  in the book "Toric Varieties" by David Cox, Hal Schenk and John Little:

Let $X_\Sigma$ be a simplicial toric variety. Then there is a bijective correspondence
$$ \{ \text{ closed subvarieties of $X_\Sigma$ } \} \leftrightarrow \{ \text{ radical homogeneous ideals $I \subseteq B(\Sigma) \subseteq S$} \} \, .$$
In this expression $B(\Sigma)$ is the irrelevant ideal. For instance, for $\mathrm{dP}_2$ we have:

In [157]:
irrelevant_ideal(dP2)

ideal(x2*x3*e2, x1*x3*e2, x2*x3*e1, x1*x2*e1, x1*e1*e2)

Consequently, for every radical homogeneous ideal $I \subseteq B(\Sigma) \subseteq S$, we find a closed subvariety $V( I ) \subseteq X_\Sigma$ and every closed subvariety of a simplicial toric variety $X_\Sigma$ arises in this way.

An algebraic cycle is a formal linear sum of such closed subvarieties. Modulo rational equivalence, the algebraic cycles furnish the Chow ring. We can compute this ring as follows in OSCAR:

In [158]:
chow_ring(dP2)

Quotient
  of multivariate polynomial ring in 5 variables x1, x2, x3, e1, e2
    over rational field
  by ideal(x1 - x3 + e1, x2 - x3 + e1 - e2, x1*x2, x1*x3, e1*e2, x2*e2, x3*e1)

There are several ways to create a rational equivalence class of an algebraic cycle. Here is one way to create such an object:

In [159]:
d = toric_divisor(dP2, [1, 2, 3, 4, 5]);
ac = rational_equivalence_class(d)

Rational equivalence class on a normal toric variety represented by 6V(x3)+V(e1)+7V(e2)

So $ac$ is represented by an algebraic cycle which in turn is given by a formal linear sum of the closed subvarieties $V(x_3)$, $V(e_1)$ and $V(e_2)$, namely $6 V(x_3) + 1 V( e_1 ) + 7 V( e_2 )$.

Intersections of rational equivalence classes can be computed in OSCAR:

In [160]:
ac * ac

Rational equivalence class on a normal toric variety represented by 34V(x2,x3)

If we only care about topological intersection numbers, then the intersection form suffices:

In [161]:
intersection_form(dP2)

Dict{MPolyRingElem, QQFieldElem} with 15 entries:
  x1*x3 => 0
  x3*e2 => 1
  x1^2  => -1
  x1*e1 => 1
  e2^2  => -1
  x3^2  => 0
  x2*x3 => 1
  e1*e2 => 0
  x2*e1 => 1
  x1*e2 => 1
  x2^2  => 0
  e1^2  => -1
  x3*e1 => 0
  x1*x2 => 0
  x2*e2 => 0

Note that this is obtained upon normalization with the volume form:

In [162]:
volume_form(dP2)

Cohomology class on a normal toric variety given by -e2^2

## 4.C Toric morphisms

Morphisms among toric varieties are well understood. Let $X$ and $Y$ be toric varieties with polyhedral fans $F_X$ in $\mathbb{Z}^n$ and $F_Y$ in $\mathbb{Z}^m$. It can be shown that a grid morphism $\mathbb{Z}^n \to \mathbb{Z}^m$, such that the image of every cone in $F_X$ is contained in a cone of $F_Y$ defines a morphism $X \to Y$. Even the converse is true!

Consequently, toric morphisms are computationally accessible and a lot is known about them. Let us give a simple example:

In [163]:
domain_variety = projective_space(NormalToricVariety, 1)

Normal toric variety

In [164]:
codomain_variety = hirzebruch_surface(2)

Normal toric variety

In [165]:
mapping_matrix = matrix(ZZ, [[0, 1]])
grid_morphism = hom(character_lattice(domain_variety), character_lattice(codomain_variety), mapping_matrix)

Map
  from GrpAb: Z
  to GrpAb: Z^2

In [166]:
tm = toric_morphism(domain_variety, grid_morphism, codomain_variety)

Toric morphism

A toric morphism induces, among others, a map among the class groups of the toric varieties in question. In the example above we have:

In [167]:
induced_morphism = morphism_on_class_group(tm)

Map
  from GrpAb: Z
  to GrpAb: Z^2

In [168]:
matrix(induced_morphism)

[0   1]

# 5 Toric schemes and subvarieties

Every toric variety has an underlying schemes, obtained by forgetting the toric origin and "just" gluing affine schemes. On the long run, we aim to extend the interplay between schemes and toric varieties:
* Toric algorithms often exhibit superior performance over generic scheme algorithms.
* Certain operations, e.g. blow-ups, can force us to leave the regime of toric geometry. This is easily facilitated by consider the toric variety as schemes

Let us return to one of our starting examples to demonstrate this "forgetting the toric origin":

In [169]:
C2 = positive_hull([1 1; -1 1])
antv2 = affine_normal_toric_variety(C2)
scheme_of_antv2 = underlying_scheme(antv2)

Spectrum
  of quotient
    of multivariate polynomial ring in 3 variables x1, x2, x3
      over rational field
    by ideal(-x1*x2 + x3^2)

Recall that we introduced affine toric varieties via theorem 1.2.18 of the book *Toric Varieties* by David Cox, John Little and Hal Schenck:

Let $\sigma \subseteq N_{\mathbb{R}} \cong \mathbb{R}^n$ be a rational polyhedral cone with semigroup $S_\sigma = \sigma^\vee \cap M$. Then the associated affine toric variety $U_\sigma$ is given by

$$ U_\sigma = \mathrm{Spec} \left(\mathbb{C} \left[ S_\sigma \right]\right) = \mathrm{Spec} \left(\mathbb{C} \left[ \sigma^\vee \cap M \right]\right) \, .$$

Above, *scheme_of_antv2* is exactly $U_\sigma$! The interested reader can find details on how to investigate this scheme $U_\sigma$ in the documentation on the scheme functionality.

Certainly, we would also like to be able to define subvarieties of toric varieties in the same way that we do for schemes/varieties. Indeed this is possible. As proven in the book *Toric varieties* by David Cox, John Little, Hal Schenk every closed subvariety of a simplicial toric variety is cut out by finitely many homogeneous polynomials in the Cox ring. Here is an example:

In [170]:
(x1, x2, x3) = gens(cox_ring(P2))
c = closed_subvariety_of_toric_variety(P2, [x1])

Closed subvariety of a normal toric variety

It can happen that such a subvariety is the empty set. We easily verify that *c* is not empty:

In [171]:
is_empty(c)

false

Here is an example of an empty subvariety:

In [172]:
c2 = closed_subvariety_of_toric_variety(P2, [x1, x2, x3])
is_empty(c2)

true

This is not at all surprising. After all, we know that the homogeneous coordinates *x1*, *x2*, *x3* of the projective space must not vanish simultaneously.

The defining ideal of such a subvariety is recovered as follows:

In [173]:
defining_ideal(c)

ideal(x1)

In [174]:
defining_ideal(c2)

ideal(x1, x2, x3)